In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0
!pip install lyricsgenius

In [2]:
!git clone https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 613 (delta 68), reused 46 (delta 24), pack-reused 505
Receiving objects: 100% (613/613), 360.66 KiB | 504.00 KiB/s, done.
Resolving deltas: 100% (368/368), done.


In [3]:
!mkdir models/

In [4]:
!git clone https://github.com/Sushentsev/PopNet.git

Cloning into 'PopNet'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 159 (delta 25), reused 145 (delta 15), pack-reused 0
Receiving objects: 100% (159/159), 8.01 MiB | 12.36 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [5]:
%cd PopNet/data

/content/PopNet/data


In [6]:
!ls

examples	      genius_examples  lyrics_collection.py
genius_collection.py  gensongs.zip


In [ ]:
!unzip gensongs.zip

# Data Collection

In [ ]:
import re

f = open('artists.txt', 'r')
artists = []

for line in f:
    if re.match('— (.*)', line) and not re.match('— (.*)( \(Верифицирован.*\))', line):
        artists.append(line.strip()[2:])

print(artists)

In [ ]:
!python genius_collection.py

In [10]:
import os

len(os.listdir('genius_examples'))

9191

In [ ]:
!zip -r gensongs.zip genius_examples

In [12]:
!git add /content/PopNet/data/genius_collection.py
!git add /content/PopNet/data/gensongs.zip

!git config --global user.email "grazder@yandex.ru"
!git config --global user.name "grazder"

!git commit -m 'songs'

!git push 

[main a4c2deb] songs
 2 files changed, 22 insertions(+), 19 deletions(-)
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 7.94 MiB | 8.97 MiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/Sushentsev/PopNet.git
   8e28dc4..a4c2deb  main -> main


# GPT

In [8]:
import os

path = '/content/PopNet/data/genius_examples/'

files = os.listdir(path)
train_files, valid_files = files[:int(len(files) * 0.8)], files[int(len(files) * 0.8):]

names = [x.split('-')[1][:-4] for x in train_files]
files = [os.path.join(path, x) for x in train_files]

train_file = open('train.txt', 'w')

for file, name in zip(files, names):
    f = open(file, "r")
    train_file.write('<startsong>\n')
    train_file.write('<songname>' + name + '\n')
    train_file.write('<songlyrics>\n')

    for line in f:
        train_file.write(line)

    train_file.write('<endsong>\n')
    f.close()

train_file.close()

names = [x.split('-')[1][:-4] for x in valid_files]
files = [os.path.join(path, x) for x in valid_files]
valid_file = open('valid.txt', 'w')

for file, name in zip(files, names):
    f = open(file, "r")
    valid_file.write('<startsong>\n')
    valid_file.write('<songname>' + name + '\n')
    valid_file.write('<songlyrics>\n')

    for line in f:
        valid_file.write(line + '<newline>')

    valid_file.write('<endsong>\n')
    f.close()

valid_file.close()

In [9]:
%cd ../../

/content


In [10]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/popnet \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

2021-05-28 10:15:16.605148: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 10:15:20 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/28/2021 10:15:20 - INFO - filelock -   Lock 139963881622992 acquired on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100% 608/608 [00:00<00:00, 482kB/s]
05/28/2021 10:15:20 - INFO - filelock -   Lock 139963881622992 released on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
05/28/2021 10:15:21 - INFO - filelock -   Lock 139963881208528 acquired on /root/.cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc344

Eval

In [11]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

np.random.seed(42)
torch.manual_seed(42)

tok = GPT2Tokenizer.from_pretrained("models/popnet")
model = GPT2LMHeadModel.from_pretrained("models/popnet").cuda()

text = "<startsong>\n<songname> Дом\n<songlyrics>\n"
inpt = tok.encode(text, return_tensors="pt")

out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
tok.decode(out[0])

OSError: ignored